In [13]:
import numpy
from keras import Sequential
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Dense
import os
import pandas as pd
import numpy as np
from time import time as t
from sklearn.metrics import classification_report
import h5py
from keras import optimizers
from keras import callbacks
import time
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from sklearn.preprocessing import StandardScaler

import ast

In [14]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import ParameterGrid


In [15]:
from keras import backend as K

def tp_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    return true_positives
    
def fp_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    return predicted_positives-true_positives

def tn_m(y_true, y_pred):
    print('y_true - 1 '+str((y_true-1)))
    false_negatives = K.sum(K.round(K.clip((y_true-1) * (y_pred-1), 0, 1)))
    return false_negatives
    
def fn_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return possible_positives-true_positives

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [16]:
numpy.random.seed(0)

In [17]:
n_splits=4
train_size=0.8
test_size=0.2

In [18]:
class EEGSeizureDatasetBalanced():
    """EEG Alco Train dataset."""

    def __init__(self):
        """
        Args:
            none.
        """
        h5f = h5py.File('seizure_scalars_unbalanced.h5','r')
        self.spikes_seizure_eeg = h5f['dataset_seizure_scalars_unbalanced'][:]
        self.spikes_seizure_eeg=np.swapaxes(self.spikes_seizure_eeg,1,2)
        scalers = {}        
        for i in range(self.spikes_seizure_eeg.shape[1]):
            scalers[i] = StandardScaler()
            self.spikes_seizure_eeg[:, i, :] = scalers[i].fit_transform(self.spikes_seizure_eeg[:, i, :]) 

        h5f.close()
        
        h5f = h5py.File('seizure_labels_unbalanced.h5','r')
        self.labels_seizure_eeg = h5f['dataset_seizure_labels_unbalanced'][:]
        print(str(np.sum(self.labels_seizure_eeg))+'/'+str(len(self.labels_seizure_eeg)))
        h5f.close()

        
    def get_data(self):
        #all folds
        dataArray = list()
        sss = StratifiedShuffleSplit(n_splits=n_splits, train_size=train_size, test_size=test_size, random_state=0)
        for train_index, test_index in sss.split(self.spikes_seizure_eeg, self.labels_seizure_eeg):
            
            
            trainLabels=self.labels_seizure_eeg[train_index]
            trainValues=self.spikes_seizure_eeg[train_index]
            testLabels=self.labels_seizure_eeg[test_index]
            testValues=self.spikes_seizure_eeg[test_index]

            #BALANCING TRAINING DATA
            positivesIndices=trainLabels==1
            positiveEEGs=trainValues[positivesIndices]
            negativeEEGs=trainValues[~positivesIndices]
            print('positiveEEGs: '+str(len(positiveEEGs)))
            print('negativeEEGs: '+str(len(negativeEEGs)))

            n=np.min([len(positiveEEGs),len(negativeEEGs)])
            print(n)

            trainValues=(np.concatenate((positiveEEGs[0:n],negativeEEGs[0:n]),axis=0))
            trainLabels=(np.concatenate((np.full((n),1),np.full((n),0)),axis=0))
            
            shuffle = np.random.RandomState(seed=0).permutation(len(trainValues))
            trainValues = trainValues[shuffle]
            trainLabels = trainLabels[shuffle]
            currentSplit = {'X_train': (trainValues), 'X_test': (testValues), 
                            'y_train': (trainLabels), 'y_test': (testLabels)}
            dataArray.append(currentSplit)
        return dataArray
    

    def __len__(self):
        return len(self.spikes_seizure_eeg)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        eeg = torch.tensor(self.spikes_seizure_eeg[idx])
        print('eeg size (in getitem): '+str(eeg.size()))
        label = self.labels_seizure_eeg[idx]
            
        sample = {'eeg': eeg, 'label': label}
        return sample

In [19]:
seizureDataset = EEGSeizureDatasetBalanced()
dataArray = seizureDataset.get_data()

100.0/500
positiveEEGs: 80
negativeEEGs: 320
80
positiveEEGs: 80
negativeEEGs: 320
80
positiveEEGs: 80
negativeEEGs: 320
80
positiveEEGs: 80
negativeEEGs: 320
80


In [20]:
starting_point = 0

In [21]:
data = 'seizure_lstm'

In [22]:
class TimeHistory(callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()
        print('epoch time start: '+str(self.epoch_time_start))

    def on_epoch_end(self, batch, logs={}):
        end_time = time.time() - self.epoch_time_start
        self.times.append(end_time)
        print('epoch time measured: '+str(end_time))



In [23]:
n_epochs=250

In [24]:
resultsFilename = str(data)+"_results_data_FINAL.txt"

if os.path.isfile(resultsFilename):
    print ("Results file exists")
else:
    print ("Results file doesn't exist, creating new file...")

overallPrecisionList = list()
overallRecallList = list()
overallAccuracyList = list()
overallF1List = list()
overallTPList = list()
overallTNList = list()
overallFPList = list()
overallFNList = list()
overallConvergenceEpochList = list()
overallTrainingTimeList = list()
overallMeanEpochTimeList = list()

with open(resultsFilename, "a") as text_file:
    print(f"Test Results :\n\n", file=text_file)
for fold in np.arange(n_splits):
    early_stopping_callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=500)
    checkpoint_callback = ModelCheckpoint('optimal_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
    currentFold = fold
    # Train the network.
    print("Begin training for fold " + str(currentFold) + "\n")
    start = t()
    # create the model
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, 
                   input_shape=(1, 4096), activation='sigmoid'))
    #model.add(Dropout(0.2))
    model.add(LSTM(128, activation='sigmoid'))
    #model.add(Dropout(0.2))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy',f1_m,precision_m, recall_m, tp_m, fp_m, tn_m, fn_m])
    print(model.summary())
    time_callback = TimeHistory()
    print('input data shape: '+str(dataArray[currentFold]['X_train'].shape))
    modelHistory=model.fit(dataArray[currentFold]['X_train'], dataArray[currentFold]['y_train'], epochs=n_epochs,
              batch_size=50, 
              validation_data=(dataArray[currentFold]['X_test'], dataArray[currentFold]['y_test']),
              callbacks=[time_callback, early_stopping_callback, checkpoint_callback])

    stoppingEpoch=early_stopping_callback.stopped_epoch
    modelTimes = time_callback.times
    
    historyF1 = modelHistory.history['val_f1_m']
    historyPrecision = modelHistory.history['val_precision_m']
    historyRecall = modelHistory.history['val_recall_m']
    historyAccuracy = modelHistory.history['val_acc']
    historyTP = modelHistory.history['val_tp_m']
    historyFP = modelHistory.history['val_fp_m']
    historyTN = modelHistory.history['val_tn_m']
    historyFN = modelHistory.history['val_fn_m']
    
    convergenceEpochs=str(historyAccuracy.index(max(historyAccuracy)))
    totalTrainingTime=np.sum(np.array(modelTimes[int(convergenceEpochs)]).astype(np.float))
    print('type(totalTrainingTime): '+str(type(totalTrainingTime)))
    print('type(convergenceEpochs): '+str(type(convergenceEpochs)))
    meanTimePerEpoch=totalTrainingTime/int(convergenceEpochs)

    with open(resultsFilename, "a") as text_file:
        print(f"Training complete for fold {str(currentFold)}\n", file=text_file)
        print(f"Epoch Times: {str(modelTimes[0:int(convergenceEpochs)])}\n", file=text_file)
        print(f"Fold {str(currentFold)} Test Metrics:\n", file=text_file)
        print(f"Fold TP: {str(historyTP)}\nFold TN: {str(historyTN)}\nFold FP: {str(historyFP)}\nFold FN: {str(historyFN)}\n", file=text_file)
        print(f"Fold Accuracy: {str(historyAccuracy)}\nFold Precision: {str(historyPrecision)}\nFold Recall: {str(historyRecall)}\nFold F1: {str(historyF1)}\n", file=text_file)
        print(f"Accuracy Maxes at Epoch: {str(convergenceEpochs)}\n", file=text_file)
        print(f"Early stopping at epoch: {str(stoppingEpoch)}\n", file=text_file)
        print(f"Training Epochs until max accuracy: {str(convergenceEpochs)}\n", file=text_file)
        print(f"Total Training Time: {str(totalTrainingTime)}\n", file=text_file)
        print(f"Mean Time per Epoch: {str(meanTimePerEpoch)}\n", file=text_file)

    # load the saved model
    saved_model = load_model('optimal_model.h5', custom_objects={"f1_m": f1_m,
                                                              "precision_m": precision_m,
                                                              "recall_m": recall_m,
                                                            "tp_m":tp_m, 
                                                             "fp_m":fp_m,
                                                             "tn_m":tn_m,
                                                             "fn_m":fn_m
                                                            })

    loss, accuracy, f1_score, precision, recall, tp, fp, tn, fn  = saved_model.evaluate(dataArray[currentFold]['X_test'], dataArray[currentFold]['y_test'], 
                            verbose=1)
    
    predictions=(saved_model.predict(dataArray[currentFold]['X_test'])>0.5).reshape(-1)
    truelabels=dataArray[currentFold]['y_test']==1
    tp=np.sum(np.logical_and(predictions, truelabels))
    tn=np.sum(np.logical_and(np.invert(predictions), np.invert(truelabels)))
    fp=np.sum(np.logical_and(predictions, np.invert(truelabels)))
    fn=np.sum(np.logical_and(np.invert(predictions), (truelabels)))
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    f1_score = (2*precision*recall)/(precision+recall)
    print(f"TP: {str(tp)}\nTN: {str(tn)}\nFP: {str(fp)}\nFN: {str(fn)}\n")
    print(f"Accuracy: {str(accuracy)}\nPrecision: {str(precision)}\nRecall: {str(recall)}\nF1: {str(f1_score)}\n")
    
    with open(resultsFilename, "a") as text_file:
        print(f"Final Test Metrics for {str(currentFold)}:\n", file=text_file)
        print(f"TP: {str(tp)}\nTN: {str(tn)}\nFP: {str(fp)}\nFN: {str(fn)}\n", file=text_file)
        print(f"Accuracy: {str(accuracy)}\nPrecision: {str(precision)}\nRecall: {str(recall)}\nF1: {str(f1_score)}\n", file=text_file)

    overallPrecisionList.append(historyPrecision)
    overallRecallList.append(historyRecall)
    overallAccuracyList.append(historyAccuracy)
    overallF1List.append(historyF1)
    overallTPList.append(historyTP)
    overallTNList.append(historyTN)
    overallFPList.append(historyFP)
    overallFNList.append(historyFN)
    overallConvergenceEpochList.append(convergenceEpochs)
    overallTrainingTimeList.append(totalTrainingTime)
    overallMeanEpochTimeList.append(meanTimePerEpoch)
    
precisionMeanOverFolds=np.average(np.array(overallPrecisionList), axis=0)
recallMeanOverFolds=np.average(np.array(overallRecallList), axis=0)
accuracyMeanOverFolds=np.average(np.array(overallAccuracyList), axis=0)
F1MeanOverFolds=np.average(np.array(overallF1List), axis=0)
TPMeanOverFolds = np.average(np.array(overallTPList), axis=0)
TNMeanOverFolds = np.average(np.array(overallTNList), axis=0)
FPMeanOverFolds = np.average(np.array(overallFPList), axis=0)
FNMeanOverFolds = np.average(np.array(overallFNList), axis=0)
convergenceMeanOverFolds = np.mean(np.array(overallConvergenceEpochList).astype(np.float))
totalTrainingTimeMeanOverFolds = np.mean(np.array(overallTrainingTimeList).astype(np.float))
EpochTimeMeanOverFolds = np.mean(np.array(overallMeanEpochTimeList).astype(np.float))


maxAccuracyIndex=int(np.argmax(accuracyMeanOverFolds))


with open(str(resultsFilename), "a") as text_file:
    print(f"Training complete for all folds.", file=text_file)
    print(f"Mean Test Metrics Over All Folds:\n", file=text_file)
    print(f"Final Accuracy: {str(accuracyMeanOverFolds[maxAccuracyIndex])}\n", file=text_file)
    print(f"Final Precision: {str(precisionMeanOverFolds[maxAccuracyIndex])}\n", file=text_file)
    print(f"Final Recall: {str(recallMeanOverFolds[maxAccuracyIndex])}\n", file=text_file)
    print(f"Final F1: {str(F1MeanOverFolds[maxAccuracyIndex])}\n", file=text_file)
    print(f"Final TP: {str(TPMeanOverFolds[maxAccuracyIndex])}\n", file=text_file)
    print(f"Final FP: {str(FPMeanOverFolds[maxAccuracyIndex])}\n", file=text_file)
    print(f"Final TN: {str(TNMeanOverFolds[maxAccuracyIndex])}\n", file=text_file)
    print(f"Final FN: {str(FNMeanOverFolds[maxAccuracyIndex])}\n", file=text_file)
    print(f"Obs. Until Max Accuracy: {str(convergenceMeanOverFolds)}\n", file=text_file)
    print(
        f"TP: {str(list(TPMeanOverFolds))}\nTN: {str(list(TNMeanOverFolds))}\nFP: {str(list(FPMeanOverFolds))}\nFN: {str(list(FNMeanOverFolds))}\n",
        file=text_file)
    print(
        f"Accuracy: {str(list(accuracyMeanOverFolds))}\nPrecision: {str(list(precisionMeanOverFolds))}\nRecall: {str(list(recallMeanOverFolds))}\nF1: {str(list(F1MeanOverFolds))}\n",
        file=text_file)

Results file exists
Begin training for fold 0

y_true - 1 Tensor("metrics_8/tn_m/sub:0", shape=(?, ?), dtype=float32)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 1, 128)            2163200   
_________________________________________________________________
lstm_10 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_9 (Dense)              (None, 30)                3870      
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 31        
Total params: 2,298,685
Trainable params: 2,298,685
Non-trainable params: 0
_________________________________________________________________
None
input data shape: (160, 1, 4096)
Train on 160 samples, validate on 100 samples
Epoch 1/250
epoch time start: 1578163350.7853427
160/160 [====

160/160 [==============================] - 0s 1ms/step - loss: 0.0072 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 23.2500 - fp_m: 0.0000e+00 - tn_m: 24.2500 - fn_m: 0.0000e+00 - val_loss: 0.2195 - val_acc: 0.9300 - val_f1_m: 0.8444 - val_precision_m: 0.8016 - val_recall_m: 0.8990 - val_tp_m: 9.0000 - val_fp_m: 2.5000 - val_tn_m: 37.5000 - val_fn_m: 1.0000
epoch time measured: 0.1647956371307373

Epoch 00030: val_acc did not improve
Epoch 31/250
epoch time start: 1578163359.969289
160/160 [==============================] - 0s 1ms/step - loss: 0.0065 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 23.5000 - fp_m: 0.0000e+00 - tn_m: 24.0000 - fn_m: 0.0000e+00 - val_loss: 0.2259 - val_acc: 0.9300 - val_f1_m: 0.8444 - val_precision_m: 0.8016 - val_recall_m: 0.8990 - val_tp_m: 9.0000 - val_fp_m: 2.5000 - val_tn_m: 37.5000 - val_fn_m: 1.0000
epoch time measured: 0.1700148582458496

Epoch 00031: val_acc did not improve
Epoch 32/25

epoch time measured: 0.17034077644348145

Epoch 00045: val_acc did not improve
Epoch 46/250
epoch time start: 1578163362.6001575
160/160 [==============================] - 0s 1ms/step - loss: 0.0030 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 23.2500 - fp_m: 0.0000e+00 - tn_m: 24.2500 - fn_m: 0.0000e+00 - val_loss: 0.2347 - val_acc: 0.9300 - val_f1_m: 0.8444 - val_precision_m: 0.8016 - val_recall_m: 0.8990 - val_tp_m: 9.0000 - val_fp_m: 2.5000 - val_tn_m: 37.5000 - val_fn_m: 1.0000
epoch time measured: 0.16822504997253418

Epoch 00046: val_acc did not improve
Epoch 47/250
epoch time start: 1578163362.768538
160/160 [==============================] - 0s 1ms/step - loss: 0.0028 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 23.5000 - fp_m: 0.0000e+00 - tn_m: 24.0000 - fn_m: 0.0000e+00 - val_loss: 0.2370 - val_acc: 0.9300 - val_f1_m: 0.8444 - val_precision_m: 0.8016 - val_recall_m: 0.8990 - val_tp_m: 9.0000 - val_fp_m: 2.500

epoch time measured: 0.1576690673828125

Epoch 00061: val_acc did not improve
Epoch 62/250
epoch time start: 1578163365.1806457
160/160 [==============================] - 0s 1ms/step - loss: 0.0014 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 24.0000 - fp_m: 0.0000e+00 - tn_m: 23.5000 - fn_m: 0.0000e+00 - val_loss: 0.2433 - val_acc: 0.9400 - val_f1_m: 0.8611 - val_precision_m: 0.8291 - val_recall_m: 0.8990 - val_tp_m: 9.0000 - val_fp_m: 2.0000 - val_tn_m: 38.0000 - val_fn_m: 1.0000
epoch time measured: 0.16640663146972656

Epoch 00062: val_acc did not improve
Epoch 63/250
epoch time start: 1578163365.3472025
160/160 [==============================] - 0s 1ms/step - loss: 0.0013 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 23.7500 - fp_m: 0.0000e+00 - tn_m: 23.7500 - fn_m: 0.0000e+00 - val_loss: 0.2434 - val_acc: 0.9400 - val_f1_m: 0.8611 - val_precision_m: 0.8291 - val_recall_m: 0.8990 - val_tp_m: 9.0000 - val_fp_m: 2.000

epoch time measured: 0.16672873497009277

Epoch 00077: val_acc did not improve
Epoch 78/250
epoch time start: 1578163367.8767102
160/160 [==============================] - 0s 1ms/step - loss: 8.9271e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 24.5000 - fp_m: 0.0000e+00 - tn_m: 23.0000 - fn_m: 0.0000e+00 - val_loss: 0.2476 - val_acc: 0.9400 - val_f1_m: 0.8611 - val_precision_m: 0.8291 - val_recall_m: 0.8990 - val_tp_m: 9.0000 - val_fp_m: 2.0000 - val_tn_m: 38.0000 - val_fn_m: 1.0000
epoch time measured: 0.16541314125061035

Epoch 00078: val_acc did not improve
Epoch 79/250
epoch time start: 1578163368.042293
160/160 [==============================] - 0s 997us/step - loss: 8.7639e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 23.7500 - fp_m: 0.0000e+00 - tn_m: 23.7500 - fn_m: 0.0000e+00 - val_loss: 0.2481 - val_acc: 0.9400 - val_f1_m: 0.8611 - val_precision_m: 0.8291 - val_recall_m: 0.8990 - val_tp_m: 9.0000 - val_f

...

epoch time measured: 0.1626596450805664

Epoch 00100: val_acc did not improve
Epoch 101/250
epoch time start: 1578163541.7787528
160/160 [==============================] - 0s 1ms/step - loss: 8.3567e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 24.0000 - fp_m: 0.0000e+00 - tn_m: 23.5000 - fn_m: 0.0000e+00 - val_loss: 0.5838 - val_acc: 0.8800 - val_f1_m: 0.7273 - val_precision_m: 0.6713 - val_recall_m: 0.8081 - val_tp_m: 8.0000 - val_fp_m: 4.0000 - val_tn_m: 36.0000 - val_fn_m: 2.0000
epoch time measured: 0.16941261291503906

Epoch 00101: val_acc did not improve
Epoch 102/250
epoch time start: 1578163541.948304
160/160 [==============================] - 0s 1ms/step - loss: 8.2040e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 22.7500 - fp_m: 0.0000e+00 - tn_m: 24.7500 - fn_m: 0.0000e+00 - val_loss: 0.5853 - val_acc: 0.8800 - val_f1_m: 0.7273 - val_precision_m: 0.6713 - val_recall_m: 0.8081 - val_tp_m: 8.0000 - val_fp

epoch time measured: 0.15694832801818848

Epoch 00131: val_acc did not improve
Epoch 132/250
epoch time start: 1578163546.862702
160/160 [==============================] - 0s 1ms/step - loss: 4.5643e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 23.7500 - fp_m: 0.0000e+00 - tn_m: 23.7500 - fn_m: 0.0000e+00 - val_loss: 0.6249 - val_acc: 0.8800 - val_f1_m: 0.7273 - val_precision_m: 0.6713 - val_recall_m: 0.8081 - val_tp_m: 8.0000 - val_fp_m: 4.0000 - val_tn_m: 36.0000 - val_fn_m: 2.0000
epoch time measured: 0.17214083671569824

Epoch 00132: val_acc did not improve
Epoch 133/250
epoch time start: 1578163547.0369933
160/160 [==============================] - 0s 1ms/step - loss: 4.5454e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 23.7500 - fp_m: 0.0000e+00 - tn_m: 23.7500 - fn_m: 0.0000e+00 - val_loss: 0.6279 - val_acc: 0.8800 - val_f1_m: 0.7273 - val_precision_m: 0.6713 - val_recall_m: 0.8081 - val_tp_m: 8.0000 - val_f

epoch time measured: 0.17020893096923828

Epoch 00162: val_acc did not improve
Epoch 163/250
epoch time start: 1578163552.115345
160/160 [==============================] - 0s 1ms/step - loss: 2.6250e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 23.5000 - fp_m: 0.0000e+00 - tn_m: 24.0000 - fn_m: 0.0000e+00 - val_loss: 0.6595 - val_acc: 0.8800 - val_f1_m: 0.7273 - val_precision_m: 0.6713 - val_recall_m: 0.8081 - val_tp_m: 8.0000 - val_fp_m: 4.0000 - val_tn_m: 36.0000 - val_fn_m: 2.0000
epoch time measured: 0.16695737838745117

Epoch 00163: val_acc did not improve
Epoch 164/250
epoch time start: 1578163552.2824526
160/160 [==============================] - 0s 1ms/step - loss: 2.5931e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 23.7500 - fp_m: 0.0000e+00 - tn_m: 23.7500 - fn_m: 0.0000e+00 - val_loss: 0.6601 - val_acc: 0.8800 - val_f1_m: 0.7273 - val_precision_m: 0.6713 - val_recall_m: 0.8081 - val_tp_m: 8.0000 - val_f

epoch time measured: 0.17393732070922852

Epoch 00193: val_acc did not improve
Epoch 194/250
epoch time start: 1578163557.3334067
160/160 [==============================] - 0s 1ms/step - loss: 1.8138e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 23.5000 - fp_m: 0.0000e+00 - tn_m: 24.0000 - fn_m: 0.0000e+00 - val_loss: 0.6840 - val_acc: 0.8800 - val_f1_m: 0.7273 - val_precision_m: 0.6713 - val_recall_m: 0.8081 - val_tp_m: 8.0000 - val_fp_m: 4.0000 - val_tn_m: 36.0000 - val_fn_m: 2.0000
epoch time measured: 0.1688060760498047

Epoch 00194: val_acc did not improve
Epoch 195/250
epoch time start: 1578163557.5023894
160/160 [==============================] - 0s 1ms/step - loss: 1.7936e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 23.7500 - fp_m: 0.0000e+00 - tn_m: 23.7500 - fn_m: 0.0000e+00 - val_loss: 0.6845 - val_acc: 0.8800 - val_f1_m: 0.7273 - val_precision_m: 0.6713 - val_recall_m: 0.8081 - val_tp_m: 8.0000 - val_f

epoch time measured: 0.15954089164733887

Epoch 00224: val_acc did not improve
Epoch 225/250
epoch time start: 1578163562.4910595
160/160 [==============================] - 0s 1ms/step - loss: 1.3174e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 23.5000 - fp_m: 0.0000e+00 - tn_m: 24.0000 - fn_m: 0.0000e+00 - val_loss: 0.7047 - val_acc: 0.8800 - val_f1_m: 0.7273 - val_precision_m: 0.6713 - val_recall_m: 0.8081 - val_tp_m: 8.0000 - val_fp_m: 4.0000 - val_tn_m: 36.0000 - val_fn_m: 2.0000
epoch time measured: 0.17020225524902344

Epoch 00225: val_acc did not improve
Epoch 226/250
epoch time start: 1578163562.661397
160/160 [==============================] - 0s 1ms/step - loss: 1.3041e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 24.0000 - fp_m: 0.0000e+00 - tn_m: 23.5000 - fn_m: 0.0000e+00 - val_loss: 0.7052 - val_acc: 0.8800 - val_f1_m: 0.7273 - val_precision_m: 0.6713 - val_recall_m: 0.8081 - val_tp_m: 8.0000 - val_f

In [25]:
f = open(resultsFilename)
accList=list()
precisionList=list()
recallList=list()
f1List=list()

for line in f:
    if(str(line[0:13])=='Fold Accuracy'):
        listStart1=line.find('[')
        listEnd1=line.find(']')
        list1=ast.literal_eval(line[listStart1:listEnd1+1])
        accList.append(list(list1))
    if(str(line[0:14])=='Fold Precision'):
        listStart2=line.find('[')
        listEnd2=line.find(']')
        list2=ast.literal_eval(line[listStart2:listEnd2+1])
        precisionList.append(list(list2))
    if(str(line[0:11])=='Fold Recall'):
        listStart3=line.find('[')
        listEnd3=line.find(']')
        list3=ast.literal_eval(line[listStart3:listEnd3+1])
        recallList.append(list(list3))
    if(str(line[0:7])=='Fold F1'):
        listStart4=line.find('[')
        listEnd4=line.find(']')
        list4=ast.literal_eval(line[listStart4:listEnd4+1])
        f1List.append(list(list4))
f.close()

In [26]:
import statistics

In [27]:
averageAccuracy = [statistics.mean(k) for k in zip(accList[0],accList[1],accList[2],accList[3])]

In [28]:
averagePrecisions=[statistics.mean(k) for k in zip(precisionList[0],precisionList[1],precisionList[2],precisionList[3])]

In [29]:
averageRecall=[statistics.mean(k) for k in zip(recallList[0],recallList[1],recallList[2],recallList[3])]

In [30]:
averageF1s=[statistics.mean(k) for k in zip(f1List[0],f1List[1],f1List[2],f1List[3])]

In [31]:
with open(resultsFilename, "a") as text_file:
    print(f"Average Accuracies List: {str(averageAccuracy)}", file=text_file)
    print(f"Average Precisions List: {str(averagePrecisions)}", file=text_file)
    print(f"Average Recalls List: {str(averageRecall)}", file=text_file)
    print(f"Average F1 List: {str(averageF1s)}", file=text_file)


